In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import hypothesis as h
import time
plt.rcParams['figure.figsize'] = 10, 5

In [ ]:
n_dimensions = 2
n_constrains = 5
n_positions = 20
min_positions = 10
n_repetitions = 100
directory = "results/ranks/"

key = "_d{}_c{}_p{}".format(n_dimensions, n_constrains, n_positions)

frame = h.get_frame(n_constrains, n_positions)
n_anchors_list = [n_dimensions + 1, 2*n_dimensions, 4*n_dimensions, 8*n_dimensions, 100*n_dimensions]
measuremets_range = list(range((n_dimensions + 1) * n_constrains, (n_dimensions + 1) * min_positions + 1))

ranks = np.zeros((len(measuremets_range), len(n_anchors_list), n_repetitions))
for a_idx, n_anchors in enumerate(n_anchors_list):
    anchors = h.get_anchors(n_anchors, n_dimensions)
    for m_idx, n_measurements in enumerate(measuremets_range):
        for r in range(n_repetitions):
            idx_a, idx_f = h.random_indexes(n_anchors, n_positions, n_measurements)
            constrains = h.get_left_submatrix(idx_a, idx_f, anchors, frame)
            ranks[m_idx, a_idx, r] = np.linalg.matrix_rank(constrains)

In [ ]:
max_rank = (n_dimensions + 1) * n_constrains
stop = 3 * max_rank
n_repetitions = ranks.shape[2]
x = np.array(measuremets_range) / max_rank

f, ax = plt.subplots()
for a_idx, n_anchors in enumerate(n_anchors_list):
    plt.plot(
        x,
        np.mean(ranks[:, a_idx, :], axis=1) / max_rank,
        label="mean rank, {} anchors".format(n_anchors),
        color="C{}".format(a_idx),
        linestyle='dashed')
    plt.step(
        x,
        np.sum(ranks[:, a_idx, :] >= max_rank, axis=1) / n_repetitions,
        label="probability, {} anchors".format(n_anchors),
        color="C{}".format(a_idx),
        where='post')
plt.xlabel("number of measurements")
plt.grid()
ax.xaxis.set_major_formatter(tck.FormatStrFormatter('%g (D+1)K'))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=1))
plt.legend()
plt.savefig(directory + "left_matrix_anchors" + key + ".pdf")
plt.show()

In [ ]:
n_dimensions = 2
n_constrains = 5
n_anchors = 3
min_positions = 5
n_repetitions = 100
directory = "results/ranks/"

n_anchors_list = [n_dimensions + 1, n_dimensions + 2, n_dimensions + 3, n_dimensions + 4, n_dimensions + 20]
n_measurements = (n_dimensions + 1) * n_constrains
n_positions_list = list(range(min_positions, 10*min_positions+1))

key = "_d{}_c{}_m{}".format(n_dimensions, n_constrains, n_measurements)

start = time.time()
ranks = np.zeros((len(n_positions_list), len(n_anchors_list), n_repetitions))
for a_idx, n_anchors in enumerate(n_anchors_list):
    anchors = h.get_anchors(n_anchors, n_dimensions)
    for p_idx, n_positions in enumerate(n_positions_list):
        for r in range(n_repetitions):
            frame = h.get_frame(n_constrains, n_positions)
            idx_a, idx_f = h.random_indexes(n_anchors, n_positions, n_measurements)
            constrains = h.get_left_submatrix(idx_a, idx_f, anchors, frame)
            ranks[p_idx, a_idx, r] = np.linalg.matrix_rank(constrains)
end = time.time()
print(end - start)

In [ ]:
max_rank = (n_dimensions + 1) * n_constrains

f, ax = plt.subplots()
for idx, n_anchors in enumerate(n_anchors_list):
    plt.plot(
        n_positions_list,
        np.mean(ranks[:, idx, :], axis=-1) / max_rank,
        label="mean rank, {} anchors".format(n_anchors),
        color="C{}".format(idx),
        linestyle='dashed')
    plt.step(
        n_positions_list,
        np.mean(ranks[:, idx, :] >= max_rank, axis=-1),
        label="probability, {} anchors".format(n_anchors),
        color="C{}".format(idx),
        where='post')
plt.xlabel("number of positions")
# plt.ylim(0)
plt.grid()
plt.legend()
plt.savefig(directory + "left_matrix_points2" + key + ".pdf")
plt.show()

In [ ]:
n_dimensions = 2
n_constrains = 5
n_anchors = 3
min_positions = 5
n_repetitions = 1000
directory = "results/ranks/"

n_anchors_list = [n_dimensions + 1]
n_measurements = (n_dimensions + 1) * n_constrains
n_positions_list = list(range(n_constrains, 10*n_constrains))

key = "_d{}_c{}_m{}".format(n_dimensions, n_constrains, n_measurements)

start = time.time()
ranks = np.zeros((len(n_positions_list), len(n_anchors_list), n_repetitions))
for a_idx, n_anchors in enumerate(n_anchors_list):
    anchors = h.get_anchors(n_anchors, n_dimensions)
    for p_idx, n_positions in enumerate(n_positions_list):
        for r in range(n_repetitions):
            frame = h.get_frame(n_constrains, n_positions)
            idx_a, idx_f = h.random_indexes(n_anchors, n_positions, n_measurements)
            constrains = h.get_left_submatrix(idx_a, idx_f, anchors, frame)
            ranks[p_idx, a_idx, r] = np.linalg.matrix_rank(constrains)
end = time.time()
print(end - start)

probabilities = [h.probability_few_anchors(n_dimensions, n_constrains, n) for n in n_positions_list]
probabilities = np.array(probabilities)

In [ ]:
max_rank = (n_dimensions + 1) * n_constrains

f, ax = plt.subplots()
idx = 0
n_anchors = n_anchors_list[idx]
mean =  np.mean(ranks[:, idx, :] >= max_rank, axis=-1)
std = np.std(ranks[:, idx, :] >= max_rank, axis=-1)
limit = h.probability_few_anchors_limit(n_dimensions, n_constrains)

plt.plot(
    n_positions_list,
    mean,
#     where='post',
    label="estimated probability")
ax.fill_between(
    n_positions_list,
    np.maximum(mean-std, 0),
    mean+std,
    alpha=0.2,
    label="estimated +/- std")
plt.plot(
    n_positions_list,
    probabilities,
#     where='post',
    label="calculated probability")
std_true = np.sqrt(probabilities - probabilities ** 2)
ax.fill_between(
    n_positions_list,
    np.maximum(probabilities - std_true, 0),
    probabilities + std_true,
    alpha=0.2,
    label="calcualted +/- std")
ax.axhline(
    limit,
    linestyle="--",
    color="g",
    label=r"limit for $positoins\rightarrow \infty$")
plt.xlabel("number of positions")
# plt.ylim(0)
plt.grid()
plt.legend()
plt.savefig(directory + "estimated_claculated_100" + key + ".pdf")
plt.show()